https://rosettacode.org/wiki/K-means%2B%2B_clustering#Python

In [6]:
from math import pi, sin, cos
from collections import namedtuple
from random import random, choice
from copy import copy
 
try:
    import psyco
    psyco.full()
except ImportError:
    pass
 

FLOAT_MAX = 1e100
 

class Point:
    __slots__ = ["x", "y", "group"]
    def __init__(self, x=0.0, y=0.0, group=0):
        self.x, self.y, self.group = x, y, group
 
 
def generate_points(npoints, radius):
    points = [Point() for _ in range(npoints)]
 
    # note: this is not a uniform 2-d distribution
    for p in points:
        r = random() * radius
        ang = random() * 2 * pi
        p.x = r * cos(ang)
        p.y = r * sin(ang)
 
    return points
 

def nearest_cluster_center(point, cluster_centers):
    """Distance and index of the closest cluster center"""
    def sqr_distance_2D(a, b):
        return (a.x - b.x) ** 2  +  (a.y - b.y) ** 2
 
    min_index = point.group
    min_dist = FLOAT_MAX
 
    for i, cc in enumerate(cluster_centers):
        d = sqr_distance_2D(cc, point)
        if min_dist > d:
            min_dist = d
            min_index = i
 
    return (min_index, min_dist)
 

def kpp(points, cluster_centers):
    cluster_centers[0] = copy(choice(points))
    d = [0.0 for _ in range(len(points))]
 
    for i in range(1, len(cluster_centers)):
        sum = 0
        for j, p in enumerate(points):
            d[j] = nearest_cluster_center(p, cluster_centers[:i])[1]
            sum += d[j]
 
        sum *= random()
 
        for j, di in enumerate(d):
            sum -= di
            if sum > 0:
                continue
            cluster_centers[i] = copy(points[j])
            break
 
    for p in points:
        p.group = nearest_cluster_center(p, cluster_centers)[0]
 
 
def lloyd(points, nclusters):
    cluster_centers = [Point() for _ in range(nclusters)]
 
    # call k++ init
    kpp(points, cluster_centers)
 
    lenpts10 = len(points) >> 10
 
    changed = 0
    while True:
        # group element for centroids are used as counters
        for cc in cluster_centers:
            cc.x = 0
            cc.y = 0
            cc.group = 0
 
        for p in points:
            cluster_centers[p.group].group += 1
            cluster_centers[p.group].x += p.x
            cluster_centers[p.group].y += p.y
 
        for cc in cluster_centers:
            cc.x /= cc.group
            cc.y /= cc.group
 
        # find closest centroid of each PointPtr
        changed = 0
        for p in points:
            min_i = nearest_cluster_center(p, cluster_centers)[0]
            if min_i != p.group:
                changed += 1
                p.group = min_i
 
        # stop when 99.9% of points are good
        if changed <= lenpts10:
            break
 
    for i, cc in enumerate(cluster_centers):
        cc.group = i
 
    return cluster_centers
 
def print_eps(points, cluster_centers, W=400, H=400):
    Color = namedtuple("Color", "r g b");
 
    colors = []
    for i in range(len(cluster_centers)):
        colors.append(Color((3 * (i + 1) % 11) / 11.0,
                            (7 * i % 11) / 11.0,
                            (9 * i % 11) / 11.0))
 
    max_x = max_y = -FLOAT_MAX
    min_x = min_y = FLOAT_MAX
 
    for p in points:
        if max_x < p.x: max_x = p.x
        if min_x > p.x: min_x = p.x
        if max_y < p.y: max_y = p.y
        if min_y > p.y: min_y = p.y
 
    scale = min(W / (max_x - min_x),
                H / (max_y - min_y))
    cx = (max_x + min_x) / 2
    cy = (max_y + min_y) / 2
 
    print ("%%!PS-Adobe-3.0\n%%%%BoundingBox: -5 -5 %d %d" % (W + 10, H + 10))
 
    print ("/l {rlineto} def /m {rmoveto} def\n" +
           "/c { .25 sub exch .25 sub exch .5 0 360 arc fill } def\n" +
           "/s { moveto -2 0 m 2 2 l 2 -2 l -2 -2 l closepath " +
           "   gsave 1 setgray fill grestore gsave 3 setlinewidth" +
           " 1 setgray stroke grestore 0 setgray stroke }def")
 
    for i, cc in enumerate(cluster_centers):
        print ("%g %g %g setrgbcolor" %
               (colors[i].r, colors[i].g, colors[i].b))
 
        for p in points:
            if p.group != i:
                continue
            print ("%.3f %.3f c" % ((p.x - cx) * scale + W / 2, (p.y - cy) * scale + H / 2))
 
        print ("\n0 setgray %g %g s" % ((cc.x - cx) * scale + W / 2, (cc.y - cy) * scale + H / 2))
 
    print ("\n%%%%EOF")
 
 
def main():
    npoints = 30000
    k = 7 # # clusters
 
    points = generate_points(npoints, 10)
    cluster_centers = lloyd(points, k)
    print_eps(points, cluster_centers)
 
 
main()

%!PS-Adobe-3.0
%%BoundingBox: -5 -5 410 410
/l {rlineto} def /m {rmoveto} def
/c { .25 sub exch .25 sub exch .5 0 360 arc fill } def
/s { moveto -2 0 m 2 2 l 2 -2 l -2 -2 l closepath    gsave 1 setgray fill grestore gsave 3 setlinewidth 1 setgray stroke grestore 0 setgray stroke }def
0.272727 0 0 setrgbcolor
292.970 149.873 c
242.793 138.649 c
267.204 151.877 c
316.308 67.319 c
311.220 51.293 c
287.525 83.919 c
274.788 142.293 c
239.270 128.774 c
289.267 149.972 c
222.365 132.636 c
264.531 61.089 c
317.651 133.103 c
277.141 136.074 c
277.546 144.174 c
246.333 125.484 c
263.017 144.390 c
281.072 90.653 c
255.991 158.006 c
279.984 19.114 c
210.899 120.417 c
228.724 60.543 c
236.813 141.327 c
368.928 101.574 c
355.458 85.982 c
243.827 39.804 c
244.936 33.459 c
380.547 122.248 c
267.290 167.713 c
226.179 57.491 c
252.553 91.477 c
265.636 115.004 c
321.898 142.467 c
286.159 100.488 c
251.138 148.076 c
227.685 139.966 c
259.659 152.542 c
302.456 68.741 c
286.980 136.260 c
241.990 103.122 c
2

165.001 381.295 c
124.103 292.501 c
147.944 244.454 c
123.604 295.069 c
153.267 370.999 c
128.668 247.114 c
157.563 256.538 c
159.224 255.441 c
113.763 265.561 c
164.506 390.505 c
85.963 356.676 c
85.284 255.233 c
91.972 248.897 c
170.074 265.186 c
167.724 387.623 c
81.761 279.988 c
80.574 295.895 c
109.707 267.804 c
129.949 245.935 c
96.837 274.098 c
92.993 288.090 c
113.062 363.082 c
93.982 304.951 c
123.339 322.626 c
174.610 288.899 c
90.195 294.938 c
136.687 334.528 c
142.232 268.292 c
118.974 368.331 c
67.376 267.669 c
169.258 269.794 c
158.633 312.902 c
118.033 322.679 c
111.697 333.386 c
169.505 279.016 c
121.193 314.088 c
143.351 362.554 c
150.988 392.428 c
165.473 288.792 c
92.144 271.029 c
123.099 302.472 c
168.164 303.995 c
72.549 335.664 c
112.332 288.082 c
150.096 279.049 c
81.787 293.401 c
86.998 329.260 c
117.729 246.642 c
103.923 317.803 c
147.034 367.275 c
107.021 352.883 c
164.946 346.580 c
119.328 303.957 c
71.694 342.271 c
170.687 312.314 c
75.338 355.615 c
89.361 2

389.301 160.596 c
331.828 147.677 c
321.046 291.512 c
351.115 226.205 c
275.876 200.343 c
323.820 236.423 c
342.983 272.886 c
285.156 217.040 c
291.662 176.766 c
322.091 155.405 c
310.177 191.372 c
334.034 187.185 c
264.729 200.241 c
268.354 184.662 c
305.579 227.664 c
306.544 247.899 c
380.009 264.744 c
289.592 235.301 c
304.905 246.415 c
370.187 276.899 c
325.516 288.955 c
359.859 217.002 c
313.959 160.193 c
291.461 164.570 c
322.275 244.781 c
334.521 221.528 c
387.871 154.509 c
373.979 161.945 c
325.161 208.132 c
320.075 209.930 c
335.967 287.127 c
308.060 191.553 c
349.764 165.571 c
376.941 207.087 c
330.526 208.300 c
389.886 259.153 c
282.136 227.645 c
290.985 214.489 c
330.424 193.945 c
285.115 202.959 c
275.094 171.477 c
267.913 216.819 c
339.172 152.041 c
297.963 206.022 c
283.538 261.440 c
275.628 217.668 c
360.014 183.936 c
369.964 228.428 c
299.544 162.267 c
282.842 249.144 c
355.403 285.275 c
336.879 187.510 c
278.370 228.143 c
382.354 230.884 c
279.633 244.912 c
283.135 17

134.491 142.672 c
212.243 6.905 c
149.249 141.471 c
119.340 77.162 c
154.307 10.133 c
94.228 68.736 c
83.535 55.445 c
137.333 147.222 c
179.707 73.087 c
179.144 127.731 c
191.646 4.856 c
124.707 110.046 c
196.966 75.950 c
122.113 21.734 c
52.257 66.424 c
167.111 54.580 c
53.773 83.125 c
157.983 146.919 c
167.716 130.541 c
86.640 91.185 c
124.606 135.385 c
160.543 15.977 c
216.764 44.544 c
144.153 119.029 c
205.134 40.134 c
150.294 130.950 c
89.069 98.604 c
128.042 125.909 c
190.728 74.807 c
105.473 57.928 c
127.395 94.693 c
186.536 27.744 c
183.699 129.903 c
107.805 25.457 c
222.347 34.406 c
69.722 73.046 c
82.325 83.285 c
187.032 64.245 c
200.766 49.118 c
128.667 71.023 c
116.389 50.576 c
189.571 33.094 c
47.171 72.382 c
158.560 6.556 c
170.646 110.184 c
204.361 2.915 c
207.083 98.329 c
103.195 113.318 c
208.327 88.745 c
162.475 68.887 c
175.939 132.679 c
198.051 67.434 c
153.992 117.193 c
97.033 100.707 c
192.254 114.592 c
164.776 63.843 c
119.211 83.310 c
77.578 55.170 c
48.450 74.3

133.953 83.902 c
131.605 79.605 c
138.736 41.073 c
198.654 67.900 c
91.048 41.263 c
202.578 109.987 c
202.513 29.743 c
153.290 79.152 c
146.075 58.440 c
187.773 98.465 c
166.142 99.088 c
173.163 50.377 c
169.872 29.494 c
147.789 137.032 c
187.742 12.464 c
100.846 40.086 c
198.624 32.861 c
77.357 85.761 c
124.408 95.154 c
167.632 137.762 c
178.857 137.949 c
63.358 78.464 c
135.013 124.030 c
131.066 61.258 c
107.686 22.802 c
132.895 103.793 c
152.727 77.031 c
154.771 26.721 c
202.754 111.261 c
99.932 78.802 c
210.747 93.373 c
88.676 50.074 c
134.121 124.802 c
193.229 54.279 c
96.269 50.226 c
92.651 101.043 c
142.525 21.176 c
121.799 39.886 c
83.390 104.477 c
133.708 63.246 c
178.435 106.414 c
122.849 69.866 c
144.593 60.316 c
178.095 36.915 c
107.427 78.281 c
213.865 69.560 c
100.244 63.119 c
177.091 45.514 c
170.247 140.062 c
120.146 63.792 c
198.264 124.861 c
215.352 44.951 c
151.579 45.318 c
95.420 71.546 c
162.535 37.063 c
199.990 117.947 c
135.218 118.042 c
160.531 112.212 c
156.547

131.317 201.901 c
41.110 193.648 c
50.769 116.239 c
11.256 244.366 c
13.212 148.299 c
31.922 137.312 c
115.235 159.354 c
36.114 128.391 c
61.604 125.776 c
54.251 122.875 c
77.752 138.321 c
79.313 217.284 c
5.723 160.929 c
105.673 202.157 c
105.379 188.351 c
25.262 270.945 c
109.963 148.985 c
112.438 163.009 c
87.800 199.594 c
18.510 149.784 c
76.131 195.880 c
105.415 193.923 c
62.315 256.945 c
63.906 241.836 c
36.131 239.223 c
24.158 170.967 c
10.906 167.450 c
136.874 155.372 c
17.744 121.151 c
54.288 195.014 c
37.688 99.478 c
63.794 232.590 c
90.723 131.925 c
46.475 212.678 c
61.051 127.255 c
113.776 144.111 c
58.588 182.681 c
54.916 197.781 c
133.244 224.283 c
102.795 227.532 c
111.506 146.281 c
102.312 220.328 c
112.891 187.135 c
25.876 179.574 c
128.691 184.678 c
53.691 152.325 c
127.774 160.617 c
98.671 134.968 c
130.818 203.217 c
74.902 176.899 c
96.125 183.031 c
33.427 173.708 c
19.168 223.491 c
133.834 163.056 c
51.946 193.656 c
97.473 225.300 c
89.650 129.588 c
88.407 162.197 

277.710 359.990 c
258.873 310.673 c
330.408 304.815 c
287.761 371.548 c
240.661 265.412 c
244.811 354.987 c
291.096 298.672 c
290.624 327.152 c
210.717 365.562 c
330.821 340.767 c
235.415 382.285 c
231.849 309.630 c
322.281 351.617 c
268.202 327.762 c
209.288 268.430 c
197.822 386.713 c
199.102 339.774 c
268.007 264.984 c
298.772 290.440 c
255.431 315.707 c
255.176 301.636 c
240.888 349.797 c
200.155 305.610 c
309.500 362.156 c
257.934 292.575 c
189.579 305.818 c
191.773 394.587 c
217.079 348.029 c
269.330 346.752 c
278.867 331.316 c
198.121 280.026 c
227.261 370.182 c
307.403 316.518 c
270.812 272.159 c
267.787 303.252 c
287.071 322.673 c
213.621 270.032 c
255.009 278.521 c
200.941 277.836 c
207.851 296.952 c
219.498 277.840 c
219.501 261.779 c
195.337 272.305 c
280.937 371.273 c
212.566 389.082 c
197.361 334.752 c
194.386 310.405 c
326.075 310.457 c
226.589 315.257 c
254.056 342.532 c
193.598 343.684 c
212.496 341.508 c
247.927 259.998 c
220.148 267.704 c
209.880 287.162 c
184.522 36

186.160 173.551 c
176.362 206.616 c
192.974 195.844 c
206.521 227.426 c
219.128 228.139 c
203.669 185.350 c
206.221 198.439 c
208.795 137.070 c
162.187 196.467 c
198.603 234.448 c
229.167 193.179 c
202.393 240.542 c
201.953 206.305 c
183.402 197.740 c
242.860 209.725 c
246.630 211.051 c
179.037 182.425 c
201.058 198.461 c
177.331 217.007 c
146.149 225.372 c
230.797 172.777 c
204.157 208.559 c
190.533 237.825 c
235.976 157.170 c
214.721 179.212 c
256.902 193.169 c
232.665 213.421 c
162.395 185.137 c
204.393 203.574 c
216.648 142.836 c
229.850 179.483 c
210.531 252.963 c
254.074 210.332 c
190.452 178.915 c
245.115 207.253 c
239.351 165.830 c
184.859 161.947 c
187.468 265.254 c
222.046 233.669 c
240.280 230.065 c
160.406 241.106 c
204.245 207.990 c
257.958 220.890 c
218.135 237.378 c
185.086 213.125 c
174.171 175.934 c
227.805 191.991 c
221.101 172.653 c
209.610 214.618 c
195.196 161.460 c
175.772 188.758 c
194.259 206.910 c
238.277 227.335 c
187.093 252.139 c
193.036 216.703 c
225.718 18

186.639 152.823 c
196.514 149.710 c
192.564 155.462 c
142.594 229.021 c
183.202 137.861 c
186.436 198.839 c
219.562 238.960 c
196.628 198.786 c
178.644 146.304 c
175.207 196.687 c
209.798 229.353 c
200.131 152.993 c
173.800 204.668 c
253.006 200.488 c
176.165 185.160 c
218.324 207.852 c
183.108 194.969 c
257.310 227.640 c
202.248 248.553 c
189.388 211.708 c
241.611 168.084 c
249.689 179.083 c
141.140 219.639 c
232.141 206.211 c
231.408 171.009 c
210.138 251.863 c
259.767 183.517 c
147.249 169.385 c
217.284 191.553 c
141.164 226.706 c
188.591 257.134 c
215.064 200.218 c
202.759 264.746 c
196.127 199.263 c
247.925 240.185 c
180.578 186.593 c
196.476 159.900 c
238.520 169.989 c
195.622 155.743 c
236.297 233.416 c
183.033 197.711 c
206.469 203.942 c
228.461 157.653 c
197.441 181.157 c
157.847 151.514 c
187.780 170.979 c
206.090 171.545 c
242.586 159.221 c
171.650 200.156 c
242.710 235.386 c
204.367 191.475 c
184.551 230.403 c
183.660 194.298 c
190.301 194.821 c
227.069 223.925 c
188.817 17